<a href="https://colab.research.google.com/github/didi64/Colab_Test/blob/main/Classifiy_Reviews1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os, sys
from google.colab import drive
ROOT = '/content/drive/My Drive/CAS_Gregi/'
drive.mount('/content/drive', force_remount=True)

nb_path = '/content/notebooks'
os.symlink(ROOT + 'CO_modules', nb_path)
sys.path.insert(0,nb_path)


Mounted at /content/drive


In [ ]:
%env NLTK_DATA=$nb_path
!pip install --target=$nb_path contractions
!python -m textblob.download_corpora

In [ ]:
!pip install --target=$nb_path autocorrect

In [7]:
import pickle
import random

def get_train_test(seed = 42):
    '''returns two dicts train, test
       both dicts have keys [1,2,3,4,5]
       train[i] is a list of reviews with score i
    '''
    random.seed(seed)
    #root = '/content/drive/My Drive/CAS_Gregi/'


    with open(ROOT + 'data/reviews_with_GPT_scores_dict.pkl','rb') as f:
        review_dict = pickle.load(f)

    with open(ROOT + 'data/bad_reviews_2000_tuples.pkl','rb') as f:
        bad_reviews = pickle.load(f)

    for review, score in bad_reviews:
        review_dict[int(score)].append(review)

    idx = list(range(1200))
    random.shuffle(idx)


    train_reviews = {k: [v[i] for i in idx[:1000]] for k,v in review_dict.items()}
    test_reviews  = {k: [v[i] for i in idx[1000:]] for k,v in review_dict.items()}

    return train_reviews, test_reviews

In [8]:
data_train, data_test = get_train_test()

In [9]:
data_test[1][0]

'The staff was incredibly rude and unhelpful. They seemed annoyed by any questions or requests, and made me feel like I was inconveniencing them.\n'

In [10]:
import contractions
for x,y in contractions.contractions_dict.items():
    x =x.replace("'", ' ')
    contractions.add(x, y)

In [33]:
from autocorrect import Speller
spell = Speller(lang='en')

In [11]:
from textblob import TextBlob
import re
import string

In [40]:
PUNCT = re.compile(r'[{}\d_\n]'.format(re.escape(string.punctuation)))

def tag_and_normalize(text, cond = lambda x: True, stopwords = ()):
    '''returns normalized text
      - removes contractions,
      - removes punctuation and numbers and _
      - spellchecks the word
      - lemmatazies the word depending on its part of speech
        for all words with a tag so that cond(tag) is True

      cond: function f(tag) -> bool
    '''
    # tagged words
    words = []
    tt = {'J': 'a', 'R': 'a', 'V':'v', 'N': 'n'}

    text = contractions.fix(text)
    text = PUNCT.sub(' ', text)
    tb = TextBlob(text).lower().correct()

    for word, tag in tb.tags:
        if word in  stopwords:
            continue


        # only keek adj, adv, and past participes like broken,
        # advertized, cleaned,... or gerund, i.e. smelling, working, bothering,...
        if cond(tag):
            word = word.lemmatize(tt.get(tag[0]))
            words.append((word, tag))


    return words


def normalize(text, counter = 0):
    counter +=1
    if counter % 100 = 0:
        print(i, end =', ')
    text = contractions.fix(text)
    text = PUNCT.sub(' ', text)
    # tb = TextBlob(text).lower().correct()
    # tb.raw
    #text = spell(text)
    return TextBlob(text).lower().correct().raw

In [16]:
res = []
cond = lambda tag: tag[0] in 'JR' or  tag in ('VBN', 'VBG')
for review in data_test[1][:10]:
    res.append(tag_and_normalize(review, cond = cond))

In [18]:
res[3][1]

('desirable', 'JJ')

In [43]:
data_test_clean = {k:[normalize(x) for x in v] for k,v in data_test.items()}

In [45]:
data_train_clean = {k:[normalize(x) for x in v] for k,v in data_train.items()}

In [46]:
train_test = (data_train_clean, data_test_clean)

In [50]:
 with open(ROOT + 'data/hotelreviews_cleaned_train_test.pkl','wb') as f:
    pickle.dump(train_test,f)